This file is just designed to test some simple code for doing the required sum numerically versus analytically. I can then write these into a python function and import into another jupyter notebook for more complicated plots.

In [1]:
# Borrow a lot of functions from Dissertation Code in 2020Iteration.
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy import integrate

In [2]:
# These functions are needed to express the sum over R and nR analytically.
def R(n,e,l,s):
    '''Unnormalized struture function for METE.
    n,e are microscopic variables.
    l are lambdas
    s are state variables, call S, N, or E'''
    return np.exp(-l[0]*n-l[1]*n*e)

def Rsum(e,l,s):
    '''Unnormalized struture function for METE, summed over n.
    e is a microscopic variable.
    l are lambdas
    s are state variables, call S, N, or E'''
    # Define exponent for lambdas 1-2
    l12 = l[0]+l[1]*e
    num = np.exp(-l12) - np.exp(-l12*(s['N']+1))
    denom = 1 - np.exp(-l12)
    return num/denom

def nRsum(e,l,s):
    '''Unnormalized struture function for METE multiplied by n, then summed over n. 
    e is a microscopic variable.
    l are lambdas
    s are state variables, call S, N, or E'''
    # Define exponent for lambdas 1-2, with n=1 since we've done sum over n already.
    l12 = l[0]+l[1]*e
    # Split up to make it easier
    num = np.exp(-l12)-(s['N']+1)*np.exp(-l12*(s['N']+1))+s['N']*np.exp(-l12*(s['N']+2))
    denom = (1-np.exp(-l12))**2
    return num/denom

def zfun(l,s):
    '''Calculate partition function for DynaMETE. 
    This function uses the analytic sum from Rsum and single integral over e.
    The integral is done with quad over log e.
    l are lambdas
    s are state variables, call S, N, or E'''
    # Return only value not error
    return integrate.quad(lambda loge: np.exp(loge)*Rsum(np.exp(loge),l,s),0,np.log(s['E']))[0]

def constraints(l,s):
    '''This function takes in lambdas and state variables and tests the constraints numerically.
    The return is an array with the percent deviation from the exact constraint.'''
    # First get the normalization
    z = zfun(l,s)
    # Now we need the sum over n and ne
    nR = integrate.quad(lambda loge: np.exp(loge)*nRsum(np.exp(loge),l,s),0,np.log(s['E']))[0]
    neR = integrate.quad(lambda loge: np.exp(loge*2)*nRsum(np.exp(loge),l,s),0,np.log(s['E']))[0]
    # Normalize
    nR /= z
    neR /= z
    # Now get the exact values
    ns = s['N']/s['S']
    es = s['E']/s['S']
    # Now return percent difference
    return np.array([(nR-ns)/ns,(neR-es)/es])

In [3]:
# METE functions
def beta_constraint(b,s):
    '''This is the beta constraint in METE with give state variables. Use this as a function call to get beta.
    Inputs s as state variables, call S, N, or Es
    Also inputs beta
    outputs beta constraint to minimize'''
    # Generate n array up to N
    narr = np.arange(s['N'])+1
    # This is the better constraint condition to use
    return np.exp(-narr*b).sum()/(np.exp(-narr*b)/narr).sum() - s['N']/s['S']
    # Old constraint
    #return b*np.log(1/(1-np.exp(-b)))-s['S']/s['N'] #Old

def mete_lambdas(s,b0=0.0001,thresh=0.05):
    '''This returns the METE lambdas for a given set of state variables.
    Inputs s as state variables, call S, N, or E
    Optional input of an initial beta, if we know it's going to be somewhere other than small positive.
    outputs array of lambdas
    Also checks to see if conditions are actually satisfied, and outputs a warning if the percent difference
    of the result is greater than 5% (by default)
    This can be modified to exclude this test by setting thresh to False, or changing it to a difference percent'''
    beta = fsolve(beta_constraint,b0,args=s)[0]
    l2 = s['S']/(s['E']-s['N']) 
    ls = np.array([beta-l2,l2]) # Only 2 compared to 5 from dynamete
    if thresh:
        con = constraints(ls,s)
        if np.any(con > thresh):
            print("Constraints are not satisfied within {:.0f}%".format(thresh*100))
    return ls

In [36]:
# Define biomass equations
def biomass(s,power=4/3):
    '''Take in state variables and numerically calculate the biomass.
    This is the equivalent of B = S Sum_n n e^(power) R 
    This function uses th exact sum over n and then does the integral over e^(power)
    The power is by default 4/3, but can be changed if we want to test other scaling relationships
    The lambdas are calculated using the function mete_lambdas above, which are approximate, 
    but should be good for most realistic state variables. The state variables should be given as a pandas Series
    s are state variables, call S, N, or E
    power is by default 4/3, but can be changed to test other scaling relationships
    '''
    # get METE lambdas
    l = mete_lambdas(s)
    # Get normalization
    z = zfun(l,s)
    # Do the required integral in log space
    b = s['S']*integrate.quad(lambda loge: np.exp((power+1)*loge)*nRsum(np.exp(loge),l,s),0,np.log(s['E']))[0]
    # Normalize and return
    return b/z

def biomass_approx(s,order=0):
    '''The approximation for the biomass equation with power=4/3'''
    # Get mete lambdas
    l = mete_lambdas(s)
    b = 4.17*s['E']**(4/3)/(s['S']**(1/3)*np.log(1/l.sum()))
    if order == 1:
        b *= (1-1.16*l.sum()**(1/3))
    return b

def biomass_approx_alt(s,order=0):
    '''The approximation for the biomass equation with power=4/3'''
    # Get mete lambdas
    l = mete_lambdas(s)
    b = 4.29*s['E']**(4/3)/(s['S']**(1/3)*np.log(1/(1-np.exp(-l.sum()))))
    if order == 1:
        b *= (1-1.16*l.sum()**(1/3)) # This gives an error when approximation doesn't hold if we use l1, so use beta
    return b

In [6]:
# Set up some state variables
bci = pd.Series([305,229000,18980000],index=['S','N','E'])
cocoli = pd.Series([154,6823,1269000],index=['S','N','E'])
sherman = pd.Series([200,8249,943400],index=['S','N','E'])
pasoh = pd.Series([717,321500,18750000],index=['S','N','E'])

In [7]:
bci_l = mete_lambdas(bci)
print(bci_l,bci_l.sum())
print(zfun(bci_l,bci))

[1.35148803e-04 1.62657992e-05] 0.00015141460246288513
540739.76896557


In [8]:
# Print numerical biomasses
print(biomass(bci))
print(biomass(cocoli))
print(biomass(sherman))
print(biomass(pasoh))
# John gets 266600000, 21920000, 10760000, 219100000

339860219.5120361
16314272.189761357
10136007.347042764
264252252.71365649


In [97]:
# Print zero order biomasses
print(biomass_approx(bci))
print(biomass_approx(cocoli))
print(biomass_approx(sherman))
print(biomass_approx(pasoh))
# John gets 266600000, 21920000, 10760000, 219100000

356591571.7073797
19440863.71688893
12194047.469050158
282629183.7628088


In [98]:
# Print first order biomasses
print(biomass_approx(bci,order=1))
print(biomass_approx(cocoli,order=1))
print(biomass_approx(sherman,order=1))
print(biomass_approx(pasoh,order=1))
# John gets 266600000, 21920000, 10760000, 219100000

334544462.6577241
15832810.10039418
9864067.001324829
261398651.49053353


In [22]:
# Testing a hypothetical gut dataset
gut = pd.Series([100,100000,200000],index=['S','N','E'])
gutl = mete_lambdas(gut)
print(gutl)
print(constraints(gutl,gut))
print(biomass(gut))
print(biomass_approx(gut)/biomass(gut))

[-0.00089034  0.001     ]
[3.30828698e-14 1.44382284e-14]
719255.5596436256
1.6022197774648073


In [33]:
# Using John's method
john_gutl = np.array([-0.000493,0.0008]) # This is what John has in the spreasheet
erange = np.arange(200000)+1
neR_gut_sum = (erange**(4/3)*nRsum(erange,john_gutl,gut)).sum()
neR_gut_int = integrate.quad(lambda loge: np.exp((4/3+1)*loge)*nRsum(np.exp(loge),gutl,gut),0,np.log(gut['E']))[0]
print(neR_gut_sum,neR_gut_int)
z_gut = Rsum(erange,john_gutl,gut).sum()
print(z_gut)
b_gut = gut['S']*neR_gut_sum/z_gut
print(b_gut)

80289287.3554087 51719493.35372385
12268.617531785934
654428.1549848025


In [38]:
# Test with actual data table I created
data = pd.read_csv('data_statevariables.txt')

In [45]:
# This is bad practice but we are doing it
# More info here:     https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
data['B'] = np.zeros(len(data))
data['B_ap'] = np.zeros(len(data))
data['B_ap_alt'] = np.zeros(len(data))
for index, row in data.iterrows():
    print(mete_lambdas(row))
    data.loc[index,'B'] = biomass(row)
    data.loc[index,'B_ap'] = biomass_approx(row,order=1)
    data.loc[index,'B_ap_alt'] = biomass_approx_alt(row,order=1)
    data.loc[index,'B_ap0'] = biomass_approx(row,order=0)
    data.loc[index,'B_ap_alt0'] = biomass_approx_alt(row,order=0)

[1.35790403e-04 1.56241996e-05]
[0.00397341 0.00012201]
[0.00425543 0.00021387]
[2.68656079e-04 3.98401209e-05]
[4.71203046e-04 3.44726967e-06]
[2.38562830e-03 8.99353664e-06]
[6.08641733e-03 3.93784118e-05]
[7.06650847e-03 3.38050409e-05]
[0.02087019 0.00193711]
[0.00916768 0.00315733]
[0.00526676 0.00295851]
[0.00811374 0.00181014]
[0.01745254 0.00354674]
[-0.00129962  0.0051716 ]


In [46]:
data

,site,S,N,E,B,B_ap,B_ap_alt,B_ap0,B_ap_alt0
0,bci,305,229000,19750000,3.585610e+08,3.527619e+08,3.629102e+08,3.760096e+08,3.868267e+08
1,cocoli,154,6823,1269000,1.631427e+07,1.583281e+07,1.628237e+07,1.944086e+07,1.999287e+07
2,sherman,200,8249,943400,1.013601e+07,9.864067e+06,1.014374e+07,1.219405e+07,1.253978e+07
3,pasoh,802,321539,20452000,2.898457e+08,2.861948e+08,2.944250e+08,3.105346e+08,3.194648e+08
4,ucsc,31,8376,9001000,3.043935e+08,2.953824e+08,3.038732e+08,3.247696e+08,3.341051e+08
5,vision,27,1844,3004000,8.718301e+07,8.434560e+07,8.675560e+07,9.984000e+07,1.026927e+08
6,bayview,16,486,406800,8.007665e+06,7.711895e+06,7.929056e+06,9.789761e+06,1.006543e+07
7,subalpine,31,877,917900,1.928997e+07,1.859771e+07,1.911919e+07,2.393389e+07,2.460499e+07
8,volcano,167,1909,88120,5.433881e+05,5.270348e+05,5.405738e+05,7.854054e+05,8.055817e+05
9,lanai,123,2253,41210,1.979027e+05,1.987506e+05,2.041842e+05,2.714987e+05,2.789210e+05


In [43]:
# Is one way better? For first order.
display((data['B_ap']-data['B'])/data['B'])
print((((data['B_ap']-data['B'])/data['B'])**2).mean())
display((data['B_ap_alt']-data['B'])/data['B'])
print((((data['B_ap_alt']-data['B'])/data['B'])**2).mean())

0    -0.016173
1    -0.029512
2    -0.026829
3    -0.012596
4    -0.029604
5    -0.032545
6    -0.036936
7    -0.035887
8    -0.030095
9     0.004285
10    0.022139
11   -0.007152
12   -0.014107
13    0.104688
dtype: float64

0.0013730395000595102


0     0.012130
1    -0.001956
2     0.000763
3     0.015799
4    -0.001709
5    -0.004902
6    -0.009817
7    -0.008853
8    -0.005179
9     0.031740
10    0.050654
11    0.020323
12    0.011520
13    0.136081
dtype: float64

0.001661910952937699


In [47]:
# For 0th order:
display((data['B_ap0']-data['B'])/data['B'])
print((((data['B_ap0']-data['B'])/data['B'])**2).mean())
display((data['B_ap_alt0']-data['B'])/data['B'])
print((((data['B_ap_alt0']-data['B'])/data['B'])**2).mean())

0     0.048663
1     0.191648
2     0.203042
3     0.071379
4     0.066940
5     0.145177
6     0.222549
7     0.240743
8     0.445386
9     0.371880
10    0.334659
11    0.322526
12    0.449910
13    0.350726
dtype: float64

0.07832652623083661


0     0.078831
1     0.225483
2     0.237152
3     0.102189
4     0.097609
5     0.177898
6     0.256975
7     0.275533
8     0.482516
9     0.409385
10    0.371892
11    0.359123
12    0.487598
13    0.389112
dtype: float64

0.09740870490205986


In [44]:
# I think this is just due to the normalization also being slightly wrong. Otherwise alt would be better
# But really there is just so much going into this approximation...
data['B'] = np.zeros(len(data))
data['B_ap'] = np.zeros(len(data))
data['B_ap_alt'] = np.zeros(len(data))
for index, row in data.iterrows():
    lt = mete_lambdas(row)
    zt = zfun(lt,row)
    ap1 = np.log(1/lt.sum())/lt[1]
    ap2 = np.log(1/(1-np.exp(-lt.sum())))/lt[1]
    print((ap1-zt)/zt)
    print((ap2-zt)/zt)
    print()

-8.607332274270978e-06
-9.099067374419648e-15

-0.0003721882422686716
-2.0983433225279753e-15

-0.0004126946593317701
1.0062081998458871e-15

-1.908028052697119e-05
-1.1904879108491886e-14

0.00047769585738414256
0.0005087204541029958

0.00018104461572544328
0.00037945090792276944

0.0020295475729437975
0.0026315885863595944

-0.0006607900273123948
5.586045507210696e-05

-0.0030015300760341404
2.322999529465086e-16

-0.0013984048639640832
6.033699826233138e-15

-0.0008553819036545446
-1.2600655334311643e-15

-0.0010736434380060743
-1.7825833938173043e-16

-0.0027057165954818647
-6.245380387189197e-16

0.01531091269531617
0.015664712426220917

